In [2]:
import ribopy
from ribopy import Ribo
from Fasta import FastaFile
from ribopy.core.get_gadgets import get_region_boundaries, get_reference_names
from functions import find_sequence, get_psite_offset, find_cds_boundaries
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

In [3]:
ribo_path   = '/Users/reikotachibana/Documents/files_analysis_c_elegans/ribo_files/Rep_5.ribo'
ribo_object = Ribo(ribo_path)
reference_path = '/Users/reikotachibana/Documents/files_analysis_c_elegans/celegans_reference/appris_celegans_v1_selected_new.fa'
# reference_path = '/Users/reikotachibana/Documents/files_analysis_c_elegans/celegans_reference/appris_celegans_v1_selected_new.fa'
# reference_path = '/Users/reikotachibana/Documents/files_analysis_c_elegans/celegans_reference/appris_celegans_v1_selected_new.fa.fai'
bed_file = '/Users/reikotachibana/Documents/files_analysis_c_elegans/celegans_reference/appris_celegans_v1_actual_regions_new.bed'
alias = False
sequence = find_sequence(ribo_object, reference_path)
exp = "WT_1cell_D"
min_len = 26
max_len = 40
offset = get_psite_offset(ribo_object, exp, min_len, max_len)
transcript_list = ribo_object.transcript_names
cds_boundaries = find_cds_boundaries(bed_file)

In [6]:
def find_coverage(ribo_object, transcript, cds_range, offset, exp, min_len, max_len, alias):
    start, stop = cds_range[transcript]
    coverages = [
        ribo_object.get_coverage(experiment=exp, range_lower=i, range_upper=i, alias=alias)
        [transcript][start - offset[i] : stop - offset[i]]
        for i in range(min_len, max_len + 1)
    ]

    if any(coverage.size == 0 for coverage in coverages):
        # If any coverage is empty, return an array of zeros
        return np.zeros_like(coverages[0])
    else:
        # Otherwise, sum the coverages
        coverage = sum(coverages, np.zeros_like(coverages[0]))
        return coverage


# Finds codon occupancy of each footprint for a transcript
def find_codon_occupancy(coverage, cds_range, sequence, transcript):
    start, stop = cds_range[transcript]
    
    # Initialize a dictionary to store the number of reads for each codon
    codon_reads = defaultdict(int)
    
    # Iterate over the range of the CDS in steps of 3 (since codons are of length 3)
    for i in range(start, stop + 1, 3):
        codon = sequence[transcript][i:i+3] # Get the codon from the sequence
        total_reads = sum(coverage[i-start:i-start+3])  # Sum up the coverage values for the codon
        codon_reads[codon] += total_reads

In [9]:
transcript = cds_boundaries.loc[1, 'Transcript']

In [24]:
len(sequence[transcript_list[1]])

1931

In [8]:
len(sequence

{'Y110A7A.10.1|cdna|chromosome:WBcel235:I:5107833:5110183:1|gene:WBGene00000001.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aap-1': 'AAGGATTCAGAATGAGCACAACACCTGGAACTCCTCATGGAGTCACTCATAGTCTTATGGAACAAGGATGGTACTGGGCTGATGCTGATAGATCTGCAGTTTCGAAAGCACTTTCTGATCAACCTGATGGATCATTTATTGTGAGAAATGCTAGTACACCTGGAGATTATACACTTTCTGTGAAATTTGCCGCCCAAGTGAAACTTCTCCGAATCGTTGTTAAAGATGGAAAATGCGGATTCAATACAGATTCATTAACTCACGATTCTGTTGTCAGATTAATTGAATTTCATCGAAATATTTCTCTCAATATTTTCAATGATGCACTTGATGTTCGTCTTTTATATCCAGTTTCTGTTCGTCGTAACTCTCAAAATGGAAAACCATTTTTCAAGAAGGGGCAACTTCAGCAGCGTATGATTCTCAGTGCAAAAAATGATCATGATTGGAGAGAACGCTTGGAAATGGAAAATTTGAGAGCAGTACATTTGGCATTTGAGAGAGGAGCCAAGCTATATGATTCAGCTCATCAAGAAATGGAACGAGCTGAAAGTCTCTATCATGCTTTGAATCAATCAGTTCGGGACAATGAGATTAAACTTGGAAAACTGAATAATTTGTTAGAAGCTGAAACAGACGTCGTTACACAAATACAGGGATCGCAGACAACAAGTGAAATGCTCAAAGGAGCATTTGCAAATAATAAGACATTTGTGGAAGAAAGTATTCGACGAATAAACACTGAACTGACGTCATCAAAGGATAAGAAAAAGACGCTCTCCGGAATTTTGGATGAAATCGCGATGAAAAGATCCAATTCAAAAACACGACTCTG

In [8]:
sumReads = 0
for transcript in transcript_list:
    coverage = find_coverage(ribo_object, transcript, cds_range, offset, exp, min_len, max_len, alias)
    sumReads += sum(coverage)

In [9]:
sumReads

3

In [37]:
transcript = 'ZK455.1.2|cdna|chromosome:WBcel235:X:11339874:11343568:1|gene:WBGene00000040.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aco-1'
start, stop = cds_range[transcript]
start, stop

(69, 162)

In [40]:
coverage = find_coverage(ribo_object, transcript, cds_range, offset, exp, min_len, max_len, alias)
coverage

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=uint16)

In [41]:
indices = np.where(coverage > 0)
indices

(array([48, 51, 80]),)

In [48]:
78%3

0

In [46]:
print(sequence[transcript][0:start])
print(sequence[transcript][start:start+48])
print(sequence[transcript][start+51:start+54])

ATATGGCTTTCAACAACCTTATCCGCAACTTGGCTATCGGAGATAACGTCTACAAATACTTTGACTTGA
ATGGTCTCAATGATGCTCGTTACAACGAGCTTCCAATCTCCATCAAGT
TGT


In [49]:
sequence[transcript][start+78:start+81]

'ATG'

In [50]:
find_codon_occupancy(coverage, cds_range, sequence, transcript)

defaultdict(int,
            {'ATG': 2,
             'GTC': 0,
             'TCA': 0,
             'CTC': 0,
             'GTT': 0,
             'ACA': 0,
             'ACG': 0,
             'AGC': 0,
             'TTC': 0,
             'CAA': 0,
             'TCT': 0,
             'CCA': 0,
             'AGT': 0,
             'ACT': 1,
             'TGT': 2,
             'TGG': 0,
             'AAG': 0,
             'CCG': 0,
             'CAG': 0,
             'ATT': 0,
             'GCG': 0,
             'TCC': 0,
             'TAT': 0,
             'TGA': 0})

In [ ]:
for transcript in transcript_list:
    coverage = ribo_object.get_coverage(experiment=exp, range_lower=26, range_upper=40, alias=alias)[transcript]
    if np.any(coverage > 0):
        print(transcript)

Y110A7A.10.1|cdna|chromosome:WBcel235:I:5107833:5110183:1|gene:WBGene00000001.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aap-1
F52H2.2b.1|cdna|chromosome:WBcel235:X:2552436:2557623:1|gene:WBGene00000004.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-3
T11F9.4a.1|cdna|chromosome:WBcel235:V:11466857:11470663:-1|gene:WBGene00000007.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-6
Y53H1C.1a.1|cdna|chromosome:WBcel235:I:11388689:11396660:-1|gene:WBGene00000010.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-9
M79.1a.1|cdna|chromosome:WBcel235:X:10605832:10624185:-1|gene:WBGene00000018.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:abl-1
ZK455.1.2|cdna|chromosome:WBcel235:X:11339874:11343568:1|gene:WBGene00000040.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aco-1
F54H12.1a.1|cdna|chromosome:WBcel235:III:

In [13]:
83%3

2

In [16]:
sequence[transcript][start+81:start+84]

'ATC'

In [61]:
33%3

0

In [ ]:
sequence[transcript][start:33+start]

In [63]:
sequence[transcript][start:stop]

'ATGGAGTCACTCATAGTCTTATGGAACAAGGATGGTACTGGGCTGATGCTGATAGATCTGCAGTTTCGAAAGCACTTTCTGATCAACCTGATGGATCATTTATTG'

In [64]:
stop - start

105

In [15]:
coverage = find_coverage(ribo_object, transcript, cds_range, offset, exp, min_len, max_len, alias)
coverage

array([0, 0, 0], dtype=uint16)

In [66]:
transcripts = ribo_object.transcript_names
cds_range = {}

# Initialize lists to store data for DataFrame creation
transcript_list = []
start_list = []
stop_list = []

# Iterate through transcripts and populate lists
for transcript in transcripts:
    start, stop = cds_range[transcript] = find_cds(sequence[transcript])
    transcript_list.append(transcript)
    start_list.append(start)
    stop_list.append(stop)

# Create a DataFrame from the lists
df = pd.DataFrame({'Transcript': transcript_list, 'Start': start_list, 'Stop': stop_list})

# Apply conditions to filter transcripts
df_filtered = df[(df['Start'].isnull()) | (df['Stop'].isnull()) | ((df['Stop'] - df['Start']) < 6)]
df_filtered

,Transcript,Start,Stop
5,C55C2.5a.1|cdna|chromosome:WBcel235:I:2571782:...,57.0,60.0
9,Y53H1C.1a.1|cdna|chromosome:WBcel235:I:1138868...,36.0,39.0
16,M79.1a.1|cdna|chromosome:WBcel235:X:10605832:1...,21.0,24.0
19,Y39D8C.1.1|cdna|chromosome:WBcel235:V:323774:3...,564.0,567.0
56,F48E3.7.1|cdna|chromosome:WBcel235:X:7503823:7...,186.0,189.0
...,...,...,...
20158,T19D2.13b.1|cdna|chromosome:WBcel235:X:3942267...,NaN,NaN
20161,R07E5.19.1|cdna|chromosome:WBcel235:III:441746...,321.0,NaN
20162,Y71H2AM.29.1|cdna|chromosome:WBcel235:III:2832...,462.0,NaN
20164,D1014.13.1|cdna|chromosome:WBcel235:V:8145629:...,396.0,NaN


In [67]:
# Save the filtered DataFrame to a CSV file
filtered_csv_filename = 'filtered_cds_ranges.csv'
df_filtered.to_csv(filtered_csv_filename, index=False)

print(f"Filtered transcripts saved to {filtered_csv_filename}")

Filtered transcripts saved to filtered_cds_ranges.csv


In [62]:
len(transcripts)

20189

In [29]:
cds_range

{'Y110A7A.10.1|cdna|chromosome:WBcel235:I:5107833:5110183:1|gene:WBGene00000001.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aap-1': (36,
  141),
 'F27C8.1.2|cdna|chromosome:WBcel235:IV:9598986:9601695:-1|gene:WBGene00000002.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-1': (129,
  351),
 'F07C3.7.1|cdna|chromosome:WBcel235:V:9244402:9246360:-1|gene:WBGene00000003.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-2': (75,
  204),
 'F52H2.2b.1|cdna|chromosome:WBcel235:X:2552436:2557623:1|gene:WBGene00000004.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-3': (0,
  1587),
 'T13A10.10a.1|cdna|chromosome:WBcel235:IV:6272529:6275713:-1|gene:WBGene00000005.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-4': (0,
  1578),
 'C55C2.5a.1|cdna|chromosome:WBcel235:I:2571782:2576020:-1|gene:WBGene00000006.1|gene_biotype:protein_coding|transcri

In [22]:
transcript = 'F39B3.7.1|cdna|chromosome:WBcel235:X:17566667:17567478:-1|gene:WBGene00304237.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:F39B3.7'
find_cds(sequence[transcript])
# find_cds(sequence[found_transcript])

(0, 396)

In [20]:
sequence[transcript]

'ATGAGCCGCTTTCGTGGCCTCTTGAAAAGTGTACTGGTCTCCTTAACAATCATTGGTACGTACCAACAAGTACAAATCAAGTGCTATGACTGTGTGACTCCACTTGGAGTGGACGATGCGACCGAGTTCTGTAATGCCAGCTTGTACTGCAAAGGGATGTACTGCACCAAAGGACCGGACGCCTTGTCAAACGGAATCTACCACGGATGCATGGACAACCCGCCAATTGATACAGCTGGAGCTACCTGCAAAGTTGTCACAAACTCCCTGGGCACTCACTCCAATTGCTTCTGCAAGAACATCGACTTTTGCAACGAGACACCAGCAGCCCGTGAGAGTCACCTATTACTCATCTTTGTGCTCATCATCCTGACCTTTTTCTACATTTTCTCACAATAA'

In [8]:

transcripts

array(['Y110A7A.10.1|cdna|chromosome:WBcel235:I:5107833:5110183:1|gene:WBGene00000001.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aap-1',
       'F27C8.1.2|cdna|chromosome:WBcel235:IV:9598986:9601695:-1|gene:WBGene00000002.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-1',
       'F07C3.7.1|cdna|chromosome:WBcel235:V:9244402:9246360:-1|gene:WBGene00000003.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:aat-2',
       ...,
       'F17A2.16.1|cdna|chromosome:WBcel235:X:12308449:12309548:1|gene:WBGene00304215.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:F17A2.16',
       'F53G2.13.1|cdna|chromosome:WBcel235:II:2485266:2485922:1|gene:WBGene00304220.1|gene_biotype:protein_coding|transcript_biotype:protein_coding|gene_symbol:F53G2.13',
       'F39B3.7.1|cdna|chromosome:WBcel235:X:17566667:17567478:-1|gene:WBGene00304237.1|gene_biotype:protein_coding|transcript_biotype:p

In [15]:
found_transcript = None
for transcript in transcripts:
    if "C25A1.8.1" in transcript:
        found_transcript = transcript
        break
        
sequence[found_transcript]

'TTTCCTCAGTCTTCACTGCTCATCATGCGTTTTTGCCTTCTCGTTGCTTTCATCCTTCCTGGGCTATTCCTCGTTCATGCAGCTCCGACTTCTTCAACCGAGCTTCCTGAAGCTAGTGGTGAAGCTCCAGAAACATCACCATTGGTTCAAAACGACGAACAGCCGCATCAACGACTGACGTTCTACAACTGGGACTACAAGGATCTCGGCACAACGGCATTCGAAGACATCTCGTTCCCAGCTCGACAGCCGCCAGCCTTCGTCAATCAGACTGAGAAGTGTCCCGATGGATGGCTTCGGTTCGCCGATTCGTGCTACTGGATCGAGAAGGAGCTTCTTGGATTCGCCAAGGCCGAGAGAAATTGCTTCGAGAAGCAGTCAACACTGTTCGTGGCCAATTCGATTGAAGAGTGGGACGCAATTCGTGTTCAAGCCAAGGAAGCCTTCTTTTCGTGGATCGGACTTGTCAGATTCACCCATTACGAGAAGTTGGAGCAGTTGCCAAGATGGCAGACTGAAGGAGCTCTCAATCCAACCAAAATCAACTGGCTCATCAAACCATACAAGCCACTCTTCAACGGTTGGTCATCACTCGCCAATTGTGCTGCCAGCTACAAATCACCATCATCCCTGGAATCGGCATCATACACGTACTTCTACCCGTGCACGTATATGCTGTACTCCATTTGTGAACGCAATTCGACCATCGTCAACGCGTTGCAGTGAAGCCTTCTGTACTTCTGACACCAATCTAATCATGTAAACAATCATCCACAACTAATCGTTTATTATTTGTCACCGGGTTCCATCCCCCTTACGTTTGACAATCATTGCACTCACTTTATAAATTTTGAGACGTTATGTATTTATATCGGAAAATAAATTTTCAACATC'

In [59]:
for i in range (0, len(sequence[found_transcript]), 3):
    codon = sequence[found_transcript][i:i+3]
    if codon == 'ATG':
        print(f'start: {i}')
    elif codon == 'TAG' or codon == 'TAA' or codon == 'TGA':
        print(f'stop: {i}')
    elif i == 722:
        print(codon)

start: 24
start: 672
stop: 723
stop: 741
stop: 879


In [6]:
# Finds coverage using p-site offset for a transcript
def find_coverage(ribo_object, transcript, cds_range, offset, exp, min_len, max_len, alias):
    start, stop = cds_range[transcript]
    coverages = [
        ribo_object.get_coverage(experiment=exp, range_lower=i, range_upper=i, alias=alias)
        [transcript][start - offset[i] : stop - offset[i]]
        for i in range(min_len, max_len + 1)
    ]

    coverage = sum(coverages, np.zeros_like(coverages[0]))
    return coverage

# Finds codon occupancy of each footprint for a transcript
def find_codon_occupancy(coverage, cds_range, nt_sequence, transcript):
    start, stop = cds_range[transcript]
    codons = [nt_sequence[transcript][start + i : start + i + 3] 
              for i in range(0, len(coverage), 3)]
    
    return Counter(codons)

In [18]:
len(sequence[found_transcript])

894

In [21]:
len(ribo_object.get_coverage(experiment="WT_1cell_D", range_lower=26, range_upper=40, alias=False)[found_transcript])

894

In [ ]:
coverage = find_coverage(ribo_object, found_transcript, cds_range, offset, exp, min_len, max_len, alias)